# Opracowanie rekurencyjnych algorytmów dla macierzy gęstych o złożoności nie większej niż O(N^3)

## Treść polecenia

Proszę wybrać ulubiony język programowania, wygenerować macierze losowe o wartościach z przedziału otwartego (0.00000001, 1.0) i zaimplementować:

- Rekurencyjne mnożenie macierzy metodą Binét’a (10 punktów)
- Rekurencyjne mnożenie macierzy metodą Strassena (10 punktów)
- Mnożenie macierzy metodą AI na podstawie artykułu w Nature (10 punktów)

Proszę zliczać liczbę operacji zmienno-przecinkowych (+-*/_liczb_) wykonywanych podczas mnożenia macierzy. \
Uwaga Wszystkie algorytmy projektowane i badane są dla macierzy o dowolnym rozmiarze, o takiej samej liczbie wierszy i kolumn (nie dotyczy macierzy mnożonych metodami AI)


#### Biblioteki

In [27]:
import numpy as np
import random

#### Generowanie losowych macierzy o wartościach z przedziału (0.00000001, 1.0)

In [28]:
# n - rozmiar macierzy
def generate_matrix(n):
    eps = np.finfo(float).eps
    return [[random.uniform(10**(-8)+eps, 1.0-eps) for _ in range(n)] for _ in range(n)]

#### Rekurencyjne mnożenie macierzy metodą Binét’a

In [ ]:
def pure_recursive_multiply(A, B):

    rows_a, cols_a = A.shape
    rows_b, cols_b = B.shape

    if cols_a != rows_b:
        raise ValueError(
            f"Wymiary się wyjebały i się nie da liczyć "
            f"liczba kolumn A ({cols_a}) musi być równa liczbie wierszy B ({rows_b})."
        )
    
    common_dim = cols_a

    if rows_a == 0 or common_dim == 0 or cols_b == 0:
        return np.zeros((rows_a, cols_b))


    if rows_a == 1 and common_dim == 1 and cols_b == 1:
        return np.array([[A[0, 0] * B[0, 0]]])

    mid_rows_a = rows_a // 2
    mid_common_dim = common_dim // 2
    mid_cols_b = cols_b // 2

    A11 = A[:mid_rows_a, :mid_common_dim]
    A12 = A[:mid_rows_a, mid_common_dim:]
    A21 = A[mid_rows_a:, :mid_common_dim]
    A22 = A[mid_rows_a:, mid_common_dim:]
    
    B11 = B[:mid_common_dim, :mid_cols_b]
    B12 = B[:mid_common_dim, mid_cols_b:]
    B21 = B[mid_common_dim:, :mid_cols_b]
    B22 = B[mid_common_dim:, mid_cols_b:]

    C11 = pure_recursive_multiply(A11, B11) + pure_recursive_multiply(A12, B21)
    C12 = pure_recursive_multiply(A11, B12) + pure_recursive_multiply(A12, B22)
    C21 = pure_recursive_multiply(A21, B11) + pure_recursive_multiply(A22, B21)
    C22 = pure_recursive_multiply(A21, B12) + pure_recursive_multiply(A22, B22)

    top_row = np.hstack((C11, C12))
    bottom_row = np.hstack((C21, C22))
    
    C = np.vstack((top_row, bottom_row))
    
    return C

In [30]:
# Funkcja generująca macierz (oryginalna i rozszerzona wersja)
def generate_square_matrix(n):
    """Generuje kwadratową macierz n x n jako listę list."""
    # Upewniamy się, że random jest zaimportowany
    import random 
    eps = np.finfo(float).eps
    return [[random.uniform(10**(-8)+eps, 1.0-eps) for _ in range(n)] for _ in range(n)]

def generate_matrix(rows, cols):
    """Generuje macierz o wymiarach rows x cols jako listę list."""
    import random
    eps = np.finfo(float).eps
    return [[random.uniform(10**(-8)+eps, 1.0-eps) for _ in range(cols)] for _ in range(rows)]


In [31]:
# ==============================================================================
# 2. Skrypt testujący
# ==============================================================================

if __name__ == "__main__":
    print("Rozpoczynanie testów poprawności algorytmu pure_recursive_multiply...\n")

    # Lista rozmiarów macierzy kwadratowych do przetestowania
    test_sizes = [1, 2, 3, 7, 8, 17]

    all_tests_passed = True

    # --- Testy dla macierzy kwadratowych ---
    for n in test_sizes:
        print(f"--- Test dla macierzy kwadratowej {n}x{n} ---")
        try:
            # Krok 1: Wygeneruj macierze jako listy list
            list_A = generate_square_matrix(n)
            list_B = generate_square_matrix(n)

            # Krok 2: Konwertuj na macierze NumPy (wymagane przez algorytm)
            A = np.array(list_A)
            B = np.array(list_B)

            # Krok 3: Wykonaj mnożenie obiema metodami
            result_recursive = pure_recursive_multiply(A, B)
            result_numpy = A @ B  # Używamy operatora @ jako "złotego standardu"

            # Krok 4: Porównaj wyniki
            # Używamy np.allclose() do porównywania liczb zmiennoprzecinkowych
            # z uwzględnieniem małych błędów precyzji.
            if np.allclose(result_recursive, result_numpy):
                print("Wynik: PASS ✅\n")
            else:
                print("Wynik: FAIL ❌\n")
                all_tests_passed = False

        except Exception as e:
            print(f"Wynik: ERROR ❌ - Wystąpił wyjątek: {e}\n")
            all_tests_passed = False
    
    # --- Test dla macierzy nieskwadratowych ---
    print(f"--- Test dla macierzy nieskwadratowych 5x3 * 3x7 ---")
    try:
        # Krok 1 i 2: Wygeneruj i skonwertuj macierze
        A = np.array(generate_matrix(5, 3))
        B = np.array(generate_matrix(3, 7))

        # Krok 3: Wykonaj mnożenie
        result_recursive = pure_recursive_multiply(A, B)
        result_numpy = A @ B

        # Krok 4: Porównaj wyniki
        if np.allclose(result_recursive, result_numpy):
            print("Wynik: PASS ✅\n")
        else:
            print("Wynik: FAIL ❌\n")
            all_tests_passed = False
            
    except Exception as e:
            print(f"Wynik: ERROR ❌ - Wystąpił wyjątek: {e}\n")
            all_tests_passed = False

    # --- Podsumowanie ---
    print("=" * 30)
    if all_tests_passed:
        print("🎉 Wszystkie testy zakończone sukcesem! 🎉")
    else:
        print("😥 Jeden lub więcej testów nie powiodło się. 😥")
    print("=" * 30)

Rozpoczynanie testów poprawności algorytmu pure_recursive_multiply...

--- Test dla macierzy kwadratowej 1x1 ---
Wynik: PASS ✅

--- Test dla macierzy kwadratowej 2x2 ---
Wynik: PASS ✅

--- Test dla macierzy kwadratowej 3x3 ---
Wynik: PASS ✅

--- Test dla macierzy kwadratowej 7x7 ---
Wynik: PASS ✅

--- Test dla macierzy kwadratowej 8x8 ---
Wynik: PASS ✅

--- Test dla macierzy kwadratowej 17x17 ---
Wynik: PASS ✅

--- Test dla macierzy nieskwadratowych 5x3 * 3x7 ---
Wynik: PASS ✅

🎉 Wszystkie testy zakończone sukcesem! 🎉


In [32]:
# ==============================================================================
# 2. Ulepszona struktura testująca
# ==============================================================================

def run_test(test_name, A, B, expected_result=None):
    """Pomocnicza funkcja do uruchamiania pojedynczego testu."""
    print(f"--- Test: {test_name} ---")
    print(f"Wymiary A: {A.shape}, Wymiary B: {B.shape}")
    
    try:
        # Jeśli nie podano oczekiwanego wyniku, oblicz go za pomocą NumPy
        if expected_result is None:
            expected_result = A @ B

        result_recursive = pure_recursive_multiply(A, B)

        if np.allclose(result_recursive, expected_result):
            print("Wynik: PASS ✅\n")
            return True
        else:
            print("Wynik: FAIL ❌ - Wyniki się nie zgadzają.\n")
            print("Oczekiwano:\n", expected_result)
            print("Otrzymano:\n", result_recursive)
            return False

    except Exception as e:
        print(f"Wynik: ERROR ❌ - Wystąpił wyjątek: {type(e).__name__}: {e}\n")
        return False

# ==============================================================================
# 3. POPRAWIONE definicje testów
# ==============================================================================

if __name__ == "__main__":
    print("Rozpoczynanie zaawansowanych testów (w tym przypadków brzegowych)...\n")
    
    results = []

    # --- Test 1: Wymiary zerowe (POPRAWIONA DEFINICJA) ---
    # Tworzymy macierze bezpośrednio w NumPy, aby zagwarantować poprawny, 2D kształt.
    A1 = np.empty((5, 0))
    B1 = np.empty((0, 7))
    results.append(run_test("Wymiar wewnętrzny równy 0", A1, B1))
    
    # --- Test 2: Jeden z wymiarów zewnętrznych równy 0 (POPRAWIONA DEFINICJA) ---
    A2 = np.empty((0, 5))
    B2 = np.array(generate_matrix(5, 3))
    results.append(run_test("Liczba wierszy w A równa 0", A2, B2))

    # --- Pozostałe testy bez zmian ---
    A3 = np.array(generate_matrix(10, 1))
    B3 = np.array(generate_matrix(1, 12))
    results.append(run_test("Wymiar wewnętrzny równy 1", A3, B3))
    
    A4 = np.array(generate_matrix(1, 10))
    B4 = np.array(generate_matrix(10, 1))
    results.append(run_test("Wymiary zewnętrzne równe 1", A4, B4))

    A5 = np.array(generate_matrix(5, 5))
    B5 = np.zeros((5, 5))
    results.append(run_test("Mnożenie przez macierz zerową", A5, B5))
    
    A6 = np.array(generate_matrix(7, 7))
    B6 = np.identity(7)
    results.append(run_test("Mnożenie przez macierz jednostkową", A6, B6, expected_result=A6))
    
    large_matrix = np.array(generate_matrix(20, 20))
    A7 = large_matrix[5:12, 3:10]
    B7 = np.array(generate_matrix(7, 7))
    results.append(run_test("Mnożenie z użyciem wycinka (slice/view)", A7, B7))

    print("--- Test: Limit głębokości rekurencji (oczekiwany błąd) ---")
    size = 64
    print(f"Wymiary A: ({size},{size}), Wymiary B: ({size},{size})")
    try:
        A8 = np.array(generate_matrix(size, size))
        B8 = np.array(generate_matrix(size, size))
        pure_recursive_multiply(A8, B8)
        print(f"Wynik: FAIL ❌ - Nie wystąpił oczekiwany błąd RecursionError.\n")
        results.append(False)
    except RecursionError:
        print(f"Wynik: PASS ✅ - Poprawnie przechwycono oczekiwany błąd RecursionError.\n")
        results.append(True)
    except Exception as e:
        print(f"Wynik: FAIL ❌ - Wystąpił inny błąd: {type(e).__name__}: {e}\n")
        results.append(False)

    # --- Podsumowanie ---
    print("=" * 40)
    if all(results):
        print(f"🎉 Wszystkie {len(results)} testów zakończone sukcesem! 🎉")
    else:
        passed_count = sum(results)
        total_count = len(results)
        print(f"😥 Testy zakończone: {passed_count}/{total_count} zakończyło się sukcesem. 😥")
    print("=" * 40)

Rozpoczynanie zaawansowanych testów (w tym przypadków brzegowych)...

--- Test: Wymiar wewnętrzny równy 0 ---
Wymiary A: (5, 0), Wymiary B: (0, 7)
Wynik: PASS ✅

--- Test: Liczba wierszy w A równa 0 ---
Wymiary A: (0, 5), Wymiary B: (5, 3)
Wynik: PASS ✅

--- Test: Wymiar wewnętrzny równy 1 ---
Wymiary A: (10, 1), Wymiary B: (1, 12)
Wynik: PASS ✅

--- Test: Wymiary zewnętrzne równe 1 ---
Wymiary A: (1, 10), Wymiary B: (10, 1)
Wynik: PASS ✅

--- Test: Mnożenie przez macierz zerową ---
Wymiary A: (5, 5), Wymiary B: (5, 5)
Wynik: PASS ✅

--- Test: Mnożenie przez macierz jednostkową ---
Wymiary A: (7, 7), Wymiary B: (7, 7)
Wynik: PASS ✅

--- Test: Mnożenie z użyciem wycinka (slice/view) ---
Wymiary A: (7, 7), Wymiary B: (7, 7)
Wynik: PASS ✅

--- Test: Limit głębokości rekurencji (oczekiwany błąd) ---
Wymiary A: (64,64), Wymiary B: (64,64)
Wynik: FAIL ❌ - Nie wystąpił oczekiwany błąd RecursionError.

😥 Testy zakończone: 7/8 zakończyło się sukcesem. 😥
